In [1]:
# import libraries 
import pandas as pd 
import numpy as np 
import os 
import re

In [ ]:
# import functions from sandbox src code files 
from raw_pose_analysis_funs.merge_mp_yolo_dfs import (merge_mp_pose_world, clean_mp_yolo_missing_data, add_orientation_and_turn_direction, save_merge_mp_yolo_df)

from raw_pose_analysis_funs.frames_to_time import (get_frames_per_second, add_time_column, save_df_w_time)

from raw_pose_analysis_funs.landmark_visibility import (mp_vis_all_labels_boxplot, mp_vis_lineplot, mp_save_vis_stats_by_label, yolo_vis_lineplot)

from raw_pose_analysis_funs.segment_video_walk_turn import filter_landmark_single_axis, segment_video_walks_turn

from raw_pose_analysis_funs.filter_interpolate_funs import (interpolate_landmark_single_axis, 
filter_landmark_single_axis) 

In [2]:
# Set input path to run all analysis 
# folder with all raw pose data (dir_in_path) 
raw_pose_data_in_path = r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\hva_code\HomeVideoAnalysis\temp\test_new_output_folders'

raw_data_full_path_all = [] 
raw_data_file_names_all = []

for (dir_path, dir_names, file_names) in os.walk(raw_pose_data_in_path):

    # file location
    
    for file_name in file_names: 
        name, ext = os.path.splitext(file_name)
        ext = ext.lower()[1:]

        # save full path to file if it meets requirements to be raw pose data 
        current_raw_data_in_path = os.path.join(dir_path, file_name)
        
        if (ext == 'csv') & ('000_raw_pose_data' in current_raw_data_in_path):
            raw_data_full_path_all = raw_data_full_path_all + [current_raw_data_in_path]
            raw_data_file_names_all = raw_data_file_names_all + [file_names]
        else: 
            print('not included')


not included
not included


In [3]:
# find all unique ID date combos 
id_date_all = []
for file_i, raw_path in enumerate(raw_data_full_path_all): 
    parent_path_1, current_file_name = os.path.split(raw_path)
    parent_path_2, current_raw_data_folder = os.path.split(parent_path_1)
    parent_path_3, current_date = os.path.split(parent_path_2)
    parent_path_4, current_id = os.path.split(parent_path_3)
    current_id_date = os.path.join(current_id, current_date)
    id_date_all = id_date_all + [current_id_date]


In [4]:
# for each unique ID and date combo, select either all gait_vertical_right .csv files or all gait_vertical_left files and run all analysis 

# all unique ID and date combos of .csv files in raw pose data folder 
unique_id_date = list(set(id_date_all))
tasks = ['gait_vertical_left', 'gait_vertical_right'] 
test_df = pd.DataFrame(data = {'A': [1, 2,3,]})

for id_date_i, id_date in enumerate(unique_id_date): 
    print(id_date_i)
    # select raw data paths that match id_date combo 
    current_id_date_raw_paths = [item for item in raw_data_full_path_all if id_date in item]

    # set output folder prefix. Find relative paths relative from input folder to raw data .csv 
    raw_data_relpath = os.path.relpath(current_id_date_raw_paths[1], raw_pose_data_in_path)
    rel_path_to_date_subfolder, raw_data_subfolder = os.path.split(os.path.dirname(raw_data_relpath))
    dir_out_prefix = os.path.normpath(os.path.join(raw_pose_data_in_path, rel_path_to_date_subfolder))

    # select raw data paths that match right or left vertical task 
    for task_i, task, in enumerate(tasks): 
        vid_in_path_no_ext = os.path.join( # only works if all videos file names are correct!! 
        current_task_raw_paths = [item for item in current_id_date_raw_paths if task in item]
        # after selecting for correct task, save each datatype path as own variable 
        current_yolo_data_path = [item for item in current_task_raw_paths if 'yolo.csv' in item]
        current_mp_pose_data_path = [item for item in current_task_raw_paths if 'mediapipe.csv' in item]
        current_mp_world_data_path = [item for item in current_task_raw_paths if 'mediapipe_world.csv' in item]
        # print(current_yolo_data_path) 
        # print(current_mp_pose_data_path) 
        # print(current_mp_world_data_path)

        # read in raw data 
        mp_pose_df = pd.read_csv(mp_pose_filepath)
        mp_world_df = pd.read_csv(mp_world_filepath)
        yolo_df = pd.read_csv(yolo_filepath)
        
        # run analysis functions 
        # step 001 - merge mp df, add tasks info and negative Y
        [mp_all_df, yolo_df] = merge_mp_pose_world(mp_pose_df, mp_world_df, yolo_df)
        [mp_all_df, yolo_df] = clean_mp_yolo_missing_data(mp_all_df, yolo_df)
        [mp_all_df, yolo_df] = add_orientation_and_turn_direction(vid_in_path, mp_all_df, yolo_df) # check vid_in_path 
        save_merge_mp_yolo_df(mp_all_df, yolo_df, vid_in_path, output_parent_folder)

# outputs 
# mp_all_df and yolo_df pandas data frames updated and .csv file saved 

        # to save outputs - os.path.join(dir_out_prefix, '001_merge', 'video_name_XYZ.csv'))
        # add vid in path variable (

        



# set output path - use os split to go back before ID, parent_path_4 from above 

0
1
